In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm import tqdm
import json
import requests
import lxml.html as lh
import pandas as pd

In [88]:
url = 'http://oir.centanet.com/retail/en/transaction?postType=rent'

In [89]:
headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
page = requests.get(url, headers=headers)

https://towardsdatascience.com/web-scraping-html-tables-with-python-c9baba21059

In [90]:
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [91]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12]

In [92]:
tr_elements = doc.xpath('//tr')

In [7]:
#Create empty list
col=[]
i=0

In [8]:
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:" "
2:"Instrument Date"
3:"District"
4:"Property"
5:"Floor"
6:"Unit"
7:"Area"
8:"Price"
9:"Price/sq.ft."
10:"Source"
11:""
12:" "


In [15]:
tr_elements[0].text_content()

'\r\n                    \xa0\r\n                    Instrument Date\r\n                    District\r\n                        Property\r\n                    Floor\r\n                    Unit\r\n                    Area\r\n                    Price\r\n                    Price/sq.ft.\r\n                    Source\r\n                    \r\n                    \xa0\r\n                '

In [46]:
tr_elements[1].text_content()

'\r\n                        \xa0\r\n                        06/11/2019\r\n\r\n                        Yuen Long\r\n                            Shop G23,G24,G59 on G/F, , Citimall, No.0001 Kau Yuk Road, Yuen Long, New Territories\r\n                        \r\n\r\n                        -\r\n\r\n                        \r\n                        -\r\n                        Approx. 1,796 sq.ft.\r\n                        For Rent $592,680\r\n                        \r\n                        \r\n                        @$330\r\n                        Market\r\n                        \r\n\r\n                        Latest Stock\r\n                        \xa0\r\n                    '

In [47]:
for t in tr_elements[1].iterchildren():
    print(t.text_content())

 
06/11/2019
Yuen Long
Shop G23,G24,G59 on G/F, , Citimall, No.0001 Kau Yuk Road, Yuen Long, New Territories
-
-
Approx. 1,796 sq.ft.
For Rent $592,680
@$330
Market
Latest Stock
 


In [19]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 12, the //tr data is not from our table 
    if len(T)!=12:
        break
    
    #k is the index of our column
    k=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if k>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[k][1].append(data)
        #Increment i for the next column
        k+=1

In [50]:
[len(C) for (title,C) in col]

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]

In [51]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [52]:
df.head()

,,Instrument Date,District,Property,Floor,Unit,Area,Price,Price/sq.ft.,Source,
0,,06/11/2019,Yuen Long,"Shop G23,G24,G59 on G/F, , Citimall, No.0001 K...",-,-,"Approx. 1,796 sq.ft.","For Rent $592,680",@$330,Market,Latest Stock
1,,06/11/2019,Tuen Mun,"Shop 17B on G/F, , Tuen Mun Centre, No.0007-00...",-,-,Approx. 500 sq.ft.,"For Rent $170,000",@$340,Market,Latest Stock
2,,06/11/2019,Tuen Mun,"Shop 059-064 on G/F, , Chi Lok Fa Yuen, No.001...",-,-,"Approx. 3,618 sq.ft.","For Rent $260,000",@$72,Market,Latest Stock
3,,06/11/2019,Central,"Shop on G/F, , No.0005 Staunton Street, Centr...",-,-,Approx. 825 sq.ft.,"For Rent $160,000",@$194,Market,Latest Stock
4,,06/11/2019,North Point,"No. 0233, City Garden, Unknown, North Point, H...",2,32,Approx. 587 sq.ft.,"For Rent $21,719",@$37,Market,Latest Stock


In [53]:
df

,,Instrument Date,District,Property,Floor,Unit,Area,Price,Price/sq.ft.,Source,
0,,06/11/2019,Yuen Long,"Shop G23,G24,G59 on G/F, , Citimall, No.0001 K...",-,-,"Approx. 1,796 sq.ft.","For Rent $592,680",@$330,Market,Latest Stock
1,,06/11/2019,Tuen Mun,"Shop 17B on G/F, , Tuen Mun Centre, No.0007-00...",-,-,Approx. 500 sq.ft.,"For Rent $170,000",@$340,Market,Latest Stock
2,,06/11/2019,Tuen Mun,"Shop 059-064 on G/F, , Chi Lok Fa Yuen, No.001...",-,-,"Approx. 3,618 sq.ft.","For Rent $260,000",@$72,Market,Latest Stock
3,,06/11/2019,Central,"Shop on G/F, , No.0005 Staunton Street, Centr...",-,-,Approx. 825 sq.ft.,"For Rent $160,000",@$194,Market,Latest Stock
4,,06/11/2019,North Point,"No. 0233, City Garden, Unknown, North Point, H...",2,32,Approx. 587 sq.ft.,"For Rent $21,719",@$37,Market,Latest Stock
5,,06/11/2019,North Point,"No. 0233, City Garden, Unknown, North Point, H...",2,33-33A,"Approx. 1,319 sq.ft.","For Rent $42,208",@$32,Market,Latest Stock
6,,06/11/2019,Kowloon Bay,"No. 0022, Kader Building, Kai Cheung Road, Kow...",2,B(PROTION),"Approx. 4,700 sq.ft.","For Rent $79,900",@$17,Market,Latest Stock
7,,06/11/2019,Wan Chai,"No. 0005-0009A, Morrison Plaza, Morrison Hill ...",8,C,Approx. 965 sq.ft.,"For Rent $27,020",@$28,Market,Latest Stock
8,,06/11/2019,Wan Chai,"No. 0005-0009A, Morrison Plaza, Morrison Hill ...",8,D,Approx. 980 sq.ft.,"For Rent $27,440",@$28,Market,Latest Stock
9,,06/11/2019,Jordan,"No. 0042, Evergreen Hotel, Woosung Street, Jor...",01-05,WF,"Approx. 22,800 sq.ft.","For Rent $800,000",@$35,Market,Latest Stock


In [26]:
df.to_csv('data.csv')

---

In [ ]:
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

In [129]:
df=pd.DataFrame()

In [153]:
page = 0
Dict={}
for i in tqdm(range(0,967)):
    page = 0+i
    url = "http://oir.centanet.com/retail/en/transaction?dept=3&priceType=total&postType=rent&transSource=0&pageIndex="+str(page)+"&pageSize=20&sortBy=5&sortDirection=2&dateRange=09%2F11%2018+2F-+09%2F11%2F2019&transDisplayMode=0&areaVerified=false"
    headers = requests.utils.default_headers()
    headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    page = requests.get(url, headers=headers)

    #Store the contents of the website under doc
    doc = lh.fromstring(page.content)
    #Parse data that are stored between <tr>..</tr> of HTML
    tr_elements = doc.xpath('//tr')
    
    #For each row, store each first element (header) and an empty list
    col=[]
    for t in tr_elements[0]:
        name=t.text_content()
        col.append((name,[]))

    #Since out first row is the header, data is stored on the second row onwards
    for j in range(1,len(tr_elements)):
        #T is our j'th row
        T=tr_elements[j]

        #If row is not of size 12, the //tr data is not from our table 
        if len(T)!=12:
            break

        #i is the index of our column
        k=0

        #Iterate through each element of the row
        for t in T.iterchildren():
            data=t.text_content() 
            #Check if row is empty
            if k>0:
            #Convert any numerical value to integers
                try:
                    data=int(data)
                except:
                    pass
            #Append the data to the empty list of the i'th column
            col[k][1].append(data)
            #Increment i for the next column
            k+=1    
        
    Dict={title:column for (title,column) in col}

#     if i == 0:
#         df=pd.DataFrame(Dict)
#     else:
#         df_temp=pd.DataFrame(Dict)
#         df=pd.concat([df, df_temp], axis =0, ignore_index=True)
        
#     if i % 10 == 0:
#         df.to_csv('data_'+str(i)+'.csv')

    if i == 0:
        df=pd.DataFrame(Dict)
    else:
        df_temp=pd.DataFrame(Dict)
        df=pd.concat([df, df_temp], axis =0, ignore_index=True)
        
    if i % 10 == 0 and i != 0:
        df.to_csv('data_page'+str(i-9)+'-'+str(i+1)+'.csv',index=False)
        df=pd.DataFrame()

100%|██████████| 967/967 [52:32<00:00,  2.61s/it]


https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

In [154]:
import pandas as pd
import glob

# path = r'C:\DRO\DCL_rawdata_files' # use your path
all_files = glob.glob("*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [168]:
frame[frame.duplicated()==True].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7156 entries, 205 to 19219
Data columns (total 11 columns):
                   7156 non-null object
Instrument Date    7156 non-null object
District           7156 non-null object
Property           7156 non-null object
Floor              7156 non-null object
Unit               7156 non-null object
Area               7156 non-null object
Price              7156 non-null object
Price/sq.ft.       7156 non-null object
Source             7156 non-null object
Unnamed: 10        7156 non-null object
dtypes: object(11)
memory usage: 670.9+ KB
